In [1]:
import numpy as np
np.random.seed(42)
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing
import os
import time
import sys

In [2]:
# In case your sys.path does not contain the base repo, go there.
print(sys.path)
%cd '/Users/axelsirota/repos/ml-solr-course'

['/Users/axelsirota/repos/ml-solr-course/3-query-generation/lab6', '/Users/axelsirota/.pyenv/versions/3.7.3/lib/python37.zip', '/Users/axelsirota/.pyenv/versions/3.7.3/lib/python3.7', '/Users/axelsirota/.pyenv/versions/3.7.3/lib/python3.7/lib-dynload', '', '/Users/axelsirota/repos/ml-solr-course/.venv/lib/python3.7/site-packages', '/Users/axelsirota/repos/ml-solr-course/.venv/lib/python3.7/site-packages/IPython/extensions', '/Users/axelsirota/.ipython']
/Users/axelsirota/repos/ml-solr-course


In [3]:
path = "dataset/train_corpus_descriptions_airbnb.csv"
# Read, then decode for py2 compat.
text = open(path, 'rb').read().decode(encoding='utf-8')[:1000000]
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')


Length of text: 1000000 characters


In [4]:
# Take a look at the first 250 characters in text
print(text[:250])


"Run Runyon Canyon, Our Gym & Sauna Open Beautifully Furnished Mirrored Mini-Suite with FireplaceDyson Hot + Cool Bladeless Fan/HeaterBeekman Goat Milk SoapPremium memory foam pillowsFirst Morning: complementary, Starbuck's coffee, latte-style coffee


In [5]:
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')


426 unique characters


In [6]:
ids_from_chars = preprocessing.StringLookup(vocabulary=list(vocab), mask_token=None)

2021-07-23 21:50:39.190649: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [27]:
chars_from_ids = preprocessing.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [8]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [9]:
ids = ids_from_chars(tf.strings.unicode_split('Only you can prevent forest fires', input_encoding='UTF-8'))
ids

<tf.Tensor: shape=(33,), dtype=int64, numpy=
array([49, 79, 77, 90,  2, 90, 80, 86,  2, 68, 66, 79,  2, 81, 83, 70, 87,
       70, 79, 85,  2, 71, 80, 83, 70, 84, 85,  2, 71, 74, 83, 70, 84])>

In [10]:
text_from_ids(ids)

<tf.Tensor: shape=(), dtype=string, numpy=b'Only you can prevent forest fires'>

In [11]:
#Prepare the dataset

all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
seq_length = 75  # Maximum alternate query size
examples_per_epoch = len(text)//(seq_length+1)
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)


In [12]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [13]:
split_input_target(list("Apache Solr"))

(['A', 'p', 'a', 'c', 'h', 'e', ' ', 'S', 'o', 'l'],
 ['p', 'a', 'c', 'h', 'e', ' ', 'S', 'o', 'l', 'r'])

In [14]:
dataset = sequences.map(split_input_target)

In [15]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'"Run Runyon Canyon, Our Gym & Sauna Open Beautifully Furnished Mirrored Min'
Target: b'Run Runyon Canyon, Our Gym & Sauna Open Beautifully Furnished Mirrored Mini'


2021-07-23 21:50:39.713969: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)


In [16]:
# Batch size
BATCH_SIZE = 64
EPOCHS = 10
BUFFER_SIZE = 2000
vocab_size = len(vocab)
embedding_dim = 100
rnn_units = 128

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
)


In [17]:
class QueryGenerator(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.rnn = tf.keras.layers.GRU(rnn_units,
                                   activation='relu',
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.rnn.get_initial_state(x)
    x, states = self.rnn(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [18]:
model = QueryGenerator(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [19]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 75, 427) # (batch_size, sequence_length, vocab_size)


In [20]:
model.summary()

Model: "query_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  42700     
_________________________________________________________________
gru (GRU)                    multiple                  88320     
_________________________________________________________________
dense (Dense)                multiple                  55083     
Total params: 186,103
Trainable params: 186,103
Non-trainable params: 0
_________________________________________________________________


In [21]:
model.compile(optimizer='adam', loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True))

In [23]:
history = model.fit(dataset, epochs=EPOCHS)

Epoch 1/10
205/205 [==============================] - 11s 49ms/step - loss: 4.7234
Epoch 2/10
205/205 [==============================] - 10s 49ms/step - loss: 3.2357
Epoch 3/10
205/205 [==============================] - 11s 55ms/step - loss: 2.8713
Epoch 4/10
205/205 [==============================] - 12s 56ms/step - loss: 2.6050
Epoch 5/10
205/205 [==============================] - 12s 55ms/step - loss: 2.4704
Epoch 6/10
205/205 [==============================] - 12s 59ms/step - loss: 2.3829
Epoch 7/10
205/205 [==============================] - 13s 60ms/step - loss: 2.3154
Epoch 8/10
205/205 [==============================] - 12s 57ms/step - loss: 2.2548
Epoch 9/10
205/205 [==============================] - 13s 61ms/step - loss: 2.2016
Epoch 10/10
205/205 [==============================] - 12s 56ms/step - loss: 2.1526


In [24]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [25]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)
start = time.time()
states = None
next_char = tf.constant(['Midtown Sunny 2-Bedroom'])
result = [next_char]

for n in range(50):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)



tf.Tensor([b'Midtown Sunny 2-Bedroomperows rov. The spaceThors, butchestwook will con '], shape=(1,), dtype=string) 

________________________________________________________________________________

Run time: 0.44038891792297363


In [26]:
tf.saved_model.save(one_step_model, '3-query-generation/lab6/alternative_queries')

2021-07-23 21:52:38.245244: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.



FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.
INFO:tensorflow:Assets written to: 3-query-generation/lab6/alternative_queries/assets
